<div style='padding:0.2em; margin:0em;
  background-color:#FBAAAA; color:#521919;
  border-left: solid #521919 4px; border-radius: 4px;'>

<p style='margin:1em; text-align:center'>
  <b>⚠ This notebook is under construction</b>
</p>

<p style='margin:1em;'>
This document is currently being worked on and is far from being complete / ready to use. And don't worry, there will be more Sage-related code in the end.
</p>

</div>

# **Fourier Transform in SageMath**
**_A project by Felix Lentze & Dominic Plein ([@Splience](https://youtube.com/@splience))_**

In this project, we will discuss what sound is, how to represent it mathematically & in a computer, and how to examine its frequency content using the Fourier Transform. The latter will be implemented in SageMath, a free open-source computer algebra system as superset of Python which bundles many great libraries like numpy, scipy, matplotlib, and more. With the help of filter matrices, the frequency spectrum of a sound can be manipulated. This is demonstrated with some sample sounds.

This projected is intended for educational purposes, i.e. we don't focus on performance of our code (for that, see the FFT algorithm implemented in system-level languages like Rust or C++). Basic knowledge of linear algebra as well as Python are required. The key points are explained next to the code. The reader is encouraged to experiment with the code and the provided sound samples.

Throughout your journey of exploring the amazing world of Fourier in this notebook, you might want to have the amazing book [Linear algebra, signal proceessing, and wavelets. A unified approach.](https://www.uio.no/studier/emner/matnat/math/nedlagte-emner/MAT-INF2360/v15/kompendium/) by Øyvind Ryan open next to you. When we reference a theorem/definition from this book, we refer to the January 21, 2015 Python edition.

<hr>

### **Preface: SageMath vs. Python?**

In the seminar, it might have come across as if Sage was a totally different programming language than Python. In fact, on the [SageMath website](https://www.sagemath.org/library-press.html), we find this:

> Leverages Existing Software: SageMath does not reinvent the wheel for every known calculation. When possible, it uses existing tools to solve the problem and combine all of them in one unique interface. This concept not only exposes software packages to a wider audience, but also helps to increase the quality by submitting bugs upstream.

> SageMath uses Python as its "glue language" to interface with all its components. Python is also SageMath's primary interface language and hence SageMath does not invent a new programming language as other mathematical software systems do. Python is well established among research communities and makes interfacing even less complicated.

So, SageMath uses Python as "glue language". In fact, it provides a unified Python interface to many great libraries in the scientific community, among other: `numpy`, `scipy`, `matplotlib`, `Maxima`, `BLAS` etc. (see also [here](https://doc.sagemath.org/html/en/faq/faq-general.html#why-did-you-write-sage-from-scratch-instead-of-using-other-existing-software-and-or-libraries)).

But then, you might wonder why you can write something like this:

```python
f(x) = x^2
```

This is not valid python syntax. You cannot name a variable f(x) in Python and to raise a number to a power, you use `**` instead of `^`. In SageMath, however, you can do this. This is because SageMath is a superset of Python, i.e. it extends the syntax of Python with some niceties like this one to make your life easier. Technically, Sage includes a [preparsing](https://doc.sagemath.org/html/en/developer/coding_in_python.html#sage-preparsing) step to convert the special syntax to valid Python code. If you want to dive deeper, [this is the Python method](https://github.com/sagemath/sage/blob/ffbbea9cb2360340f1c5517b590883fdaf02575c/src/sage/repl/preparse.py#L1382) responsible to parse statements like the one above.

In [1]:
# Only possible in Sage (but just syntactic sugar)
f(x) = x^2
print(f"Via SageMath syntax: {f(5)}")

# Sage will convert the above snippet to the following code internally.
# You can find this out by executing `preparse("f(x) = x^2")`.
__tmp__ = var("x");
f = symbolic_expression(x^Integer(2)).function(x)
print(f"Directly via Python (this is what SageMath does under the hood): {f(5)}")

Via SageMath syntax: 25
Directly via Python (this is what SageMath does under the hood): 25


**SageMath is a superset of Python. Therefore, every Python code you write is also SageMath code by definition (but not the other way around as seen in the example)**. Sage bundles many great open-source libraries and provides a unified interface to them.

### **Import packages**

From the [Sage documentation](https://doc.sagemath.org/html/en/thematic_tutorials/numerical_sage/numerical_tools.html): <small>(highlights & line breaks by us)</small>

> Sage has many different components that may be useful for numerical analysis. In particular three packages deserve mention, they are numpy, SciPy, and cvxopt.
> - **Numpy** is an excellent package that provides fast array facilities to python. It includes some basic linear algebra routines, vectorized math routines, random number generators, etc. It supports a programming style similar to one would use in matlab and most matlab techniques have an analogue in numpy.
> - **SciPy** builds on numpy and provides many different packages for optimization, root finding, statistics, linear algebra, interpolation, FFT and dsp tools, etc.

In [2]:
# Packages that ship with SageMath
# see https://doc.sagemath.org/html/en/reference/spkg/ for all packages
# that come distributed with Sage
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt

# Interact with Jupyter notebooks, e.g. show sliders
# See https://doc.sagemath.org/html/en/reference/repl/sage/repl/ipython_kernel/interact.html
from sage.repl.ipython_kernel.interact import interact

# Play audio samples in a Jupyter Notebook
from IPython.display import Audio

<hr>

### **Intro: What is sound really?**

We hear sounds every day - the chirping of birds, the rustling of leaves, music, people talking. Physically speaking, sound is a pressure wave that propagates through a medium like air. Variations in air pressure near our ears cause our eardrums to vibrate. Through some more processes, these vibrations are converted into electrical signals that are sent to our brain, which interprets these signals as sound. The frequency (number of variations per seconds) of these pressure waves determines the pitch of the sound we hear. The amplitude of the pressure wave determines the loudness of the sound. As an example, consider the following sine wave and experiment with its parameters to see how they affect the sound:

$$
\boxed{
    A: \mathbb{R} \rightarrow \mathbb{R}, \quad
    t \mapsto A(t) \coloneqq A_0 \cdot \sin(2\pi f \cdot t + \varphi)
}
$$

$$
A_0: \text{Amplitude}, \quad
f: \text{Frequency}, \quad
\varphi: \text{Phase}
$$

**Watch out for your ears when playing the sound!**

In [3]:
# Don't worry about the sample rate yet, we'll cover what it is later
# when we talk about discrete signals.
SAMPLE_RATE = float(41400)  # sample rate (samples per second, Hz)
T_sound = 1  # duration of sound (seconds)

In [4]:
@interact
def sin_wave(f=slider(0, 1400, 1, default=440),
        phi=slider(0,2*pi), A_0=slider(0,1,0.1, default=0.8)):

    # Plot the sine wave
    A(t) = A_0 * sin(2*pi*f*t + phi)
    show(plot(A(t), (t, 0, 0.06),
            color="red", thickness=1.5),
            xmin=0, xmax=0.06, ymin=-1, ymax=1,
            figsize=(10, 4), axes_labels=["t", "A(t)"])

    # Generate audio file
    t_linspace = np.linspace(0, T_sound, int(T_sound*SAMPLE_RATE), endpoint=False)
    wave = A_0 * np.sin(2*np.pi*f*t_linspace + phi)
    display(Audio(wave, rate=SAMPLE_RATE, normalize=False))

Interactive function <function sin_wave at 0x7efe11a22660> with 3 widgets
  f: TransformIntSlider(value=440, description='f', max=1400)
  phi: TransformFloatSlider(value=0.0, description='phi', max=6.283185307179586)
  A_0: TransformFloatSlider(value=0.8, description='A_0', max=1.0)

This sine wave describes the variations of air pressure, which forms the tone we hear. The bigger the frequency $f$, the higher the pitch. The bigger the amplitude $A_0$, the louder the sound.

It's therefore reasonable to say that we can model sounds by functions. A sound with frequency $f$ is modeled by a trigonometric function of that frequency. We call $\sin(2\pi f t)$ the **pure tone** of frequency $f$.

<link rel="stylesheet" href="./styles.css">
<div class="def">

<strong>Observation (Ryan, 1.1).
Sound as mathematical function.</strong>

A sound can be represented by a mathematical function, with time as the free variable.<br>When a *function* represents a sound, it is often referred to as a *continuous sound*.
<br><br>
</div>

#### **Summing up multiple sine waves**

An interesting idea might be to sum up multiple sine waves of different frequencies. We will sum them up pointwise, i.e. add up their respective values at each point in time. For example, consider the following function:

$$
\begin{align*}
A(t) &\coloneqq \sum_{n=0}^{N}
\frac{4}{(2n+1)\pi} \sin\left(2\pi (2n+1) \frac{t}{T}\right)\\

&= \frac{4}{\pi} \sin\left(2\pi \cdot \frac{t}{T}\right)
+ \frac{4}{3\pi} \sin\left(2\pi \cdot 3 \frac{t}{T}\right)
+ \frac{4}{5\pi} \sin\left(2\pi \cdot 5\frac{t}{T}\right)
+ \frac{4}{7\pi} \sin\left(2\pi \cdot 7 \frac{t}{T}\right)
+ \cdots 
\end{align*}
$$

In [ ]:
@interact
def square_wave(N=slider(2, 12, 2, default=2), auto_update=False):

    # Construct square wave
    T = 0.5  # period of square wave: 2
    t = np.linspace(0, T, 500)  # period of square wave: 2
    y = np.zeros(np.shape(t))
    for n in range(1, N, 2):
        y = y + 4 / (n * pi) * np.sin(2 * pi * 500 * n * t / T)

    # Plot square wave
    plt.figure(figsize=(10, 4))
    plt.plot(t, y, color="red", linewidth=1.5)
    plt.xlabel("t")
    plt.ylabel("A(t)")

    # Generate audio file
    # SAMPLE_RATE = float(41)  # sample rate (samples per second, Hz)
    t_linspace = np.linspace(0, T_sound, int(T_sound * SAMPLE_RATE), endpoint=False)
    wave = np.zeros(np.shape(t_linspace))
    for n in range(1, N, 2):
        wave = wave + 4 / (n * pi) * np.sin(2 * pi * 500 * n * t_linspace / T_sound)
    display(Audio(wave, rate=SAMPLE_RATE))

We observe that any reasonable function (whatever that means...) can be written as sum of simple sin- and cos- functions with integer frequencies.

<link rel="stylesheet" href="./styles.css">
<div class="def">

<strong>Observation (Ryan, 1.8).
Decomposition of sound into pure tones.</strong>

Any sound $f$ is a sum of pure tones at different frequencies. The amount of each frequency required to form $f$ is the *frequency content* of $f$. Any sound can be reconstructed from its frequency content.
<br><br>
</div>

<hr>

**TODO: Work more intensely on the following sections.**

<hr>

> The focus in Discrete Fourier analysis is to change coordinates from standard basis to the Fourier basis, performing some operations on this "Fourier representation", and then change coordinates back to the standard basis. [Ryan, p. 51]

### **Discrete Fourier Transform (DFT)**

Importing important libarys and implementing DFT function 

In [ ]:
def dft(x):
    N = len(x)
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(-2j * np.pi * k * n / N)

    X = np.dot(e, x)

    return X

Importing .wav-file and printing length of the vector and sample rate

Vector containing air pressure differences at different (discrete) time steps
Sample rate says how many samples per second the .wav-file contains (normally 44100)

In [ ]:
sample_rate, wav_data = wavfile.read("../data/sounds/sin02.wav")
print(len(wav_data))
print(wav_data)
print(sample_rate)

### **Fourier transformation**

performing DFT on wav-Vector. 
dft_result contains "frequencies" ( = periodic functions with given frequency) and quantity of frequence in measured signal

In [ ]:
dft_result = np.fft.fft(wav_data)
print(dft_result)
n = len(dft_result)
n

Determine occuring frequencies

In [ ]:
n = len(wav_data)
print(n)
freqs = np.fft.fftfreq(n)
print(freqs)

### **Plot**

Creating plot with amplitude of frequencies

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(freqs, np.real(dft_result))  # Nur die positive Frequenzhälfte
plt.title("Fouriertransformierte des WAV-Vektors")
plt.xlabel("Frequenz (Hz)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()

### **Inverse Discrete Fourier Transform (IDFT)**


In [ ]:
def idft(x):
    N = len(x)
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(2j * np.pi * k * n / N)

    X = np.dot(e, x)

    return (1/N)*X

In [ ]:
idft_result = idft(dft_result)
len(idft_result)
idft_result

### **Fix rounding issues**

In [ ]:
idft_rounded = np.around(idft_result, 6)  # 6 -> ok, 7 -> not ok

if np.array_equal(wav_data, idft_rounded):
    print("ok")
else:
    print("not ok")

### **`.wav` output**

In [ ]:
wavfile.write("../data/sounds/output.wav", 44100, idft_rounded)